In [1]:
import random
import os
import sys
import time
import numpy as np
import meshplot as mp
import import_ipynb
import igl
import matplotlib.pyplot as plt
import meshcut
from quad_mesh_simplify import simplify_mesh
from scipy.sparse.linalg import spsolve as solve
from random import random


import partitioning
import watermarking
import distortion

importing Jupyter notebook from partitioning.ipynb
importing Jupyter notebook from watermarking.ipynb
importing Jupyter notebook from embedding.ipynb
importing Jupyter notebook from extraction.ipynb
importing Jupyter notebook from distortion.ipynb


In [2]:
def embed_and_extract(patches = 60,strength = 0.1,data = [0, 1]*32):
    capacity = embed(patches,strength)
    errors,rms,hausdorff = extract(patches,strength)
    return errors,rms,hausdorff,capacity
  

In [3]:
def embed(patches = 60,strength = 0.1,data = [0, 1]*32):

    original_filename = "source_models/venus.obj"
    watermarked_filename = "watermarked_models/venus_strength"+str(strength)+ "_patches" + str(patches) + ".obj"

    # Insertion of the watermark
    embed_start = time.time()
    capacity = watermarking.embed(original_filename, watermarked_filename, data, 123456, strength, patches)
    embed_end = time.time()
    
    print(capacity," bits inserted")
    print(" ")
    print("Time to embed watermark: ", embed_end - embed_start)
    print(" ")
    print(" ")
    
    return capacity

In [4]:
def extract(patches = 60,strength = 0.1,data = [0, 1]*32):
    
    original_filename = "source_models/venus.obj"
    watermarked_filename = "watermarked_models/venus_strength"+str(strength)+ "_patches" + str(patches) + ".obj"
    #watermarked_filename = "watermarked_models/athena_transformed.obj"
    
     # Extraction of the watermark
    extract_start = time.time()
    retrieved = watermarking.extract(watermarked_filename, 123456, 64, strength, patches,original_filename)
    extract_end = time.time()
    
    print("Time taken to embed watermark: ", extract_end - extract_start)
    print(" ")
    
    # Compute the number of errors
    errors = 0
    for i in range(len(data)):
        if data[i] != retrieved[i]:
            errors +=1
    print("number of errors: ", errors)
    print(" ") 
    
    original_mesh = igl.read_triangle_mesh(original_filename)
    watermarked_mesh = igl.read_triangle_mesh(watermarked_filename)
    
    mp.plot(watermarked_mesh[0],watermarked_mesh[1])
    
    rms = distortion.rms_error(original_mesh, watermarked_mesh)
    print("RMS Error Computed: ",rms)
    print(" ")
    
    hausdorff = distortion.hausdorff_distance(original_mesh, watermarked_mesh)
    print("Hausdorff Distance Computed: ", hausdorff)
    print(" ")

    print("Original Mesh")
    mp.plot(original_mesh[0],original_mesh[1])
    print(" ")
    
    print("Watermarked Mesh")
    mp.plot(watermarked_mesh[0],watermarked_mesh[1])
    
    return errors,rms,hausdorff

In [5]:
def noise_attack(patches = 20,strength = 5,amplitude = 0.001):
    
    data = [0, 1]*32
    
    
    
    in_name = "source_models/athena.obj"
    filename = "watermarked_models/athena_strength"+str(strength)+ "_patches" + str(patches) + ".obj"
    original_mesh = igl.read_triangle_mesh(in_name)

    watermarked_mesh = igl.read_triangle_mesh(filename)
    watermarkedVertices = watermarked_mesh[0]
    watermarkedFaces = watermarked_mesh[1]
    
    file_noisy = "watermarked_models/athena_strength"+str(strength)+ "_patches" + str(patches) + "_amplitude" + str(amplitude) + ".obj"
    new_vertices = []
    for vertice in mesh[0]:
        new_vertices.append([vertice[0] + vertice[0] * random() * amplitude, vertice[1] + vertice[1] * random() * amplitude, vertice[2] + vertice[2] * random() * amplitude])
    noise_mesh = [np.array(new_vertices), mesh[1]]
    mp.plot(noise_mesh[0],noise_mesh[1])
    igl.write_triangle_mesh(file_noisy,noise_mesh[0],noise_mesh[1])
    # Extraction of the watermark
    retrieved = watermarking.extract(file_noisy, 123456, 64, strength, patches,in_name)
    errors = len([x for x, y in zip(data, retrieved) if x != y])
    print("Number of errors: ",errors)
    return errors

In [6]:
def smooth_attack(patches = 60,strength = 100,smoothness = 1):
    
    data = [0, 1]*32
    
    
    in_name = "source_models/venus.obj"
    filename = "watermarked_models/venus_strength"+str(strength)+ "_patches" + str(patches) + ".obj"
    original_mesh = igl.read_triangle_mesh(in_name)

    watermarked_mesh = igl.read_triangle_mesh(filename)
    watermarkedVertices = watermarked_mesh[0]
    watermarkedFaces = watermarked_mesh[1]
       
    file_smoothed = "watermarked_models/venus_strength"+str(strength)+ "_patches" + str(patches) + "_smoothness" + 
    str(smoothness) + ".obj"
    
    l = - igl.cotmatrix(watermarked_mesh[0],watermarked_mesh[1])
    m =   igl.massmatrix(watermarked_mesh[0],watermarked_mesh[1])
    new_vertices = solve(m + smoothness*l, m@watermarked_mesh[0])

    mp.plot(new_vertices,watermarked_mesh[1])
    igl.write_triangle_mesh(file_smoothed, new_vertices,watermarked_mesh[1])
    retrieved = watermarking.extract(file_smoothed, 123456, 64, strength, patches,in_name)
    errors = len([x for x, y in zip(data, retrieved) if x != y])
    print("Number of errors: ",errors)
    return errors

SyntaxError: invalid syntax (<ipython-input-6-308baf6fe57c>, line 14)

In [ ]:
def simplify_attack(patches = 60,strength = 0.1,simplestrength = 0.1):
    data = [0, 1]*32

    in_name = "source_models/venus.obj"
    filename = "watermarked_models/venus_strength"+str(strength)+ "_patches" + str(patches) + ".obj"
    
    original_mesh = igl.read_triangle_mesh(in_name)

    watermarked_mesh = igl.read_triangle_mesh(filename)
    watermarkedVertices = watermarked_mesh[0]
    watermarkedFaces = watermarked_mesh[1]
       
    file_simplified = "watermarked_models/venus_strength"+str(strength)+ "_patches" + str(patches) + "_simple strength" + str(simplestrength) + ".obj"
    new_vertices, new_faces = simplify_mesh(watermarked_mesh[0],watermarked_mesh[1].astype('uint32'),round(len(watermarked_mesh[1])*simplestrength))
    mp.plot(new_vertices,new_faces)
    new_faces = new_faces.astype('int32')
    igl.write_triangle_mesh(file_simplified, new_vertices, new_faces)
    ok = igl.read_triangle_mesh(file_simplified)
    retrieved = watermarking.extract(file_simplified, 123456, 64, strength, patches,in_name)
    errors = len([x for x, y in zip(data, retrieved) if x != y])
    print("Number of errors: ",errors)
    
    return errors

In [ ]:
#simplify_attack(9,0.25,0.9)

In [ ]:
if __name__ == '__main__':
    
    errorlist = []
    rmslist = []
    hauslist = []
    capacitylist = []
    modellist = ["elephant","athena","venus","bunny","hand"]
    

    mesh = igl.read_triangle_mesh("source_models/venus.obj")
    num_submeshes = round(len(mesh[0])/500)
    strengths = [x * 0.01 for x in range(29,29)]

    for i in strengths:
        errors,rms,hausdorff, capacity= embed_and_extract(num_submeshes,i)
        errorlist.append(errors)
        rmslist.append(rms)
        hauslist.append(hausdorff)
        capacitylist.append(capacity)

    x = strengths        
    y = errorlist
    plt.title("Number of errors by strength") 
    plt.xlabel("Strength") 
    plt.ylabel("Errors") 
    plt.plot(x,y)
    plt.show()

    x = strengths        
    y = rmslist
    plt.title("RMS Error by strength") 
    plt.xlabel("Strength") 
    plt.ylabel("RMS Error") 
    plt.plot(x,y)
    plt.show()

    x = strengths        
    y = hauslist
    plt.title("Hausdorff Distance by strength") 
    plt.xlabel("Strength") 
    plt.ylabel("Hausdorff Distance") 
    plt.plot(x,y)
    plt.show()

    x = strengths        
    y = capacitylist
    plt.title("Capacity by strength") 
    plt.xlabel("Strength") 
    plt.ylabel("Capacity (bits)") 
    plt.plot(x,y)
    plt.show()
    
    smoothlist = [x * 0.1 for x in range(1, 31)]
    smooth_errors = []
    for i in smoothlist:
        smooth_errors.append(smooth_attack(num_submeshes,0.02,i))

    x = smoothlist       
    y = smooth_errors
    plt.title("errors by smoothness") 
    plt.xlabel("Smoothness") 
    plt.ylabel("errors") 
    plt.plot(x,y)
    plt.show()


    simplestrengths = [x *  0.1 for x in range(1, 11)]
    simple_errors = []
    for i in simplestrengths:
        simple_errors.append(simplify_attack(num_submeshes,0.09,i))

    x = simplestrengths        
    y = simple_errors
    plt.title("errors by simplestrength") 
    plt.xlabel("simplestrength") 
    plt.ylabel("errors") 
    plt.plot(x,y)
    plt.show()

    amplitudelist = [x * 0.004 for x in range(1, 21)]
    noise_errors = []
    for i in amplitudelist:
        noise_errors.append(noise_attack(num_submeshes,0.09,i))

    x = amplitudelist        
    y = noise_errors
    plt.title("errors by amplitude") 
    plt.xlabel("Amplitude") 
    plt.ylabel("errors") 
    plt.plot(x,y)
    plt.show()

  

        

        





        
    
